In [92]:
#import libraries
import os
import re
import bs4
import csv
import time
import requests
import json, codecs
from pprint import pprint
from github import Github
from bs4 import BeautifulSoup

In [93]:
def write_json(page_objects):
    with open('data1.json','a') as f:
        json.dump(page_objects,f)

In [94]:
#define functions 
def find_tags(tags,each_issue):
    
    for counter,temp in enumerate(each.find_all('a')):
        tag = str(temp.string)
        tags.append(tag)

In [95]:
#maybe embody it in the find desc function to reduce iterations.
def remove_special_ch(text):
    
    for i in range(0,len(text)):
        text[i] = text[i].replace('\n','')
        text[i] = text[i].replace('\t','')
        text[i] = re.sub(r"<.*?>","",text[i])
        #maybe better solution is <[^<>]+> matches any character except < or > 
        #one or more times included inside < and > 

In [96]:
def find_desc(each,base_url,description,stack_trace):
    
    next_url     = base_url+each.find('a')['href']
    response     = requests.get(next_url)
    html_content = response.content
    dom          = BeautifulSoup(html_content,'html.parser')
    temp         = dom.find('div',class_ = 'edit-comment-hide')
    temp_2       = temp.find('td')
    
    #find description and the stack trace
    calculate_desc(temp_2,description,stack_trace)
    
    #find the person who closed the issue
    who_closed_it = "none"
    temp_3        = dom.find_all('div',class_='TimelineItem-body') 
    for i in temp_3:
        if i.text.find('closed this')!=-1:
            if i.find('a',class_='author Link--primary text-bold') is None:
                continue
            who_closed_it = i.find('a',class_='author Link--primary text-bold').text
    
    #remove special characters from the description and stack trace
    remove_special_ch(description)
    if stack_trace != []:
        remove_special_ch(stack_trace)
    
    return who_closed_it

In [97]:
def calculate_desc(html_text,description,stack_trace):
    
    html_content = html_text.contents
    length = len(html_content)
    
    if length == 1:
        #some lines are empty. No need to write them
        if html_content[0]!= []:
            str_temp = str(html_content[0])
            if str_temp.find('java.')!=-1 or str_temp.find('AndroidRuntime:')!=-1:
                stack_trace.append(str(html_content[0]))
            else:
                description.append(str(html_content[0]))
    else:
        for i in range(length):

            if type(html_content[i]) is bs4.element.NavigableString:
                #avoid writting empty lines and html tags <br>
                if len(html_content[i])>4:
                    description.append(str(html_content[i]))
            elif type(html_content[i] is bs4.element.Tag):
                #call recursively the function till length is 1
                calculate_desc(html_content[i],description,stack_trace)
                

In [98]:
#search all closed issues

#define some variables
total_issues  = 0
total_traces  = 0
issues        = []


#initialize important url's
base_url = "https://github.com/"
query_url = f"https://github.com/cgeo/cgeo/issues?page=1&q=is%3Aissue+is%3Aclosed"
#query_url = f"https://github.com/cgeo/cgeo/issues?page=115&q=is%3Aissue+is%3Aclosed"

#authorization in order to make more requests.
token = os.getenv("GITHUB_TOKEN")
headers = {'Authorization': f'token {token}'}

response = requests.get(query_url, headers = headers)

In [99]:
#loop through all pages

while True:
    response_code = response.status_code
    if response_code != 200:
        raise Exception("Error Occured")
    else:
        html_content = response.content
        dom = BeautifulSoup(html_content,'html.parser')
        
        #find all issues in every page
        all = dom.findAll('div', class_='flex-auto min-width-0 p-2 pr-3 pr-md-2')
        
        page_objects = []
        
        #real scraping begins
        for each in all:
            
            #find tags and who open the issue
            tags=[]
            find_tags(tags,each)
            
            
            #find description, stack trace and who closed
            description   = []
            stack_trace   = []
            who_closed_it = find_desc(each,base_url,description,stack_trace)
            
            total_issues = total_issues+1
            if stack_trace !=[]:
                total_traces = total_traces +1
            
            print(total_issues,tags[0])
            #print("********************")
            #print(description)
            #print("********************")
            #print(stack_trace)
            
            #write dictionary
            if len(tags)>=1:    
                issue_object = {'name':tags[0],'tags':tags[1:len(tags)-1],'opened_by':tags[len(tags)-1],
                               'description':description,'stack_trace':stack_trace,'closed_by':who_closed_it}
           
            page_objects.append(issue_object)
            
        #combine all issues to a global list
        issues.extend(page_objects)
        
        #debug 
        if total_issues >=25:
            break
        ####
        
        #visit the next page if exists
        end = dom.findAll('a',class_='next_page')
        
        if end ==[]:
            break
        next_url = base_url+end[1]['href']
        response = requests.get(next_url)
        
        # Sleep for 60 seconds
        time.sleep(60)

1 Default folder for themes should be below maps folder
2 Problem posting log
3 Active circles not shown on change of map type
4 Circle color setup effective only after restart (OSM)
5 Failure in LogTemplateProviderTest.testNoNumberIncrement
6 c:geo not starting after restore of data
7 Crash after renaming theme file
8 Stuck on cache refresh/delete
9 Include debug logging into SAF procedures
10 Perform theme folder sync as background task
11 Hide unittest folder selection in Settings - System
12 Theme options not working for zipped themes
13 Downloaded theme not set as current theme
14 Unresponsive/Slow app when buffering themes on SD
15 Check on existing final coords before navigation is started
16 [Nightly] Syc'ing themes folder to internal storage does not work
17 Meaning of string folder_process_status_currentfile
18 SAF: Picture attached via Google Fotos gets attached twice
19 replace com.yqritc.recyclerviewflexibledivider
20 SAF: Behavior of migration wizard if folder selection i

In [100]:
#store all issues
write_json(issues)

#print some statistics
print(total_traces)
print(total_issues)

4
25


5888


In [ ]:
#add in scraping extra for finding stack traces in issues    
#remove \n \t with regular expression etc.                   done

#this is last
#tokenization the text.
#and then create embeddings.

#not this onestack traces to graph -> (networkx)
#this one for sure !!!stack traces to dictionary             

#saturday must have finish them